## Interest Graph

In [195]:
# pip install python-twitter -> run that command in your remote machine's terminal window to install the Twitter wrapper
import twitter

# Twitter API keys go here
CONSUMER_KEY = ''
CONSUMER_SECRET = ''

OAUTH_TOKEN = ''
OAUTH_TOKEN_SECRET = ''


# let's create an instance of the twitter.Api with our credentials
api = twitter.Api(consumer_key=CONSUMER_KEY,
                  consumer_secret=CONSUMER_SECRET,
                  access_token_key=OAUTH_TOKEN,
                  access_token_secret=OAUTH_TOKEN_SECRET
)

print api

In [73]:
sn = 'gilgul'

# helps with pagination - returns next_cursor, previous_cursor, and data

f = api.GetFriendIDsPaged(screen_name=sn)

In [74]:
f

(0,
 0,
 [202503479,
  16984977,
  71738480,
  1201496244,
  212557033,
  25697700,
  13036062,
  222865265,
  17750760,
  755794526,
  2467685286,
  15880764,
  1191207924,
  879521,
  134731227,
  10717092,
  27259037,
  495245391,
  262595592,
  2207665903,
  104189196,
  163735378,
  18508860,
  14065609,
  638773,
  14363893,
  193335930,
  36443,
  20882240,
  1731561564,
  41216269,
  358933161,
  3382,
  53604599,
  98073161,
  2243034713,
  15319103,
  2282723228,
  69133574,
  1222855860,
  8515852,
  29648173,
  802953780,
  2159593434,
  11518572,
  16141659,
  108209516,
  47240812,
  718083185012244484,
  28490614,
  282168978,
  22159709,
  130745589,
  105895323,
  3321571363,
  893301,
  14807047,
  18372260,
  16948477,
  472398242,
  11679892,
  205569142,
  400636259,
  323009920,
  16243515,
  2981968107,
  18370732,
  46245868,
  239439036,
  15129265,
  471550563,
  4128127113,
  245384422,
  17163609,
  376618837,
  3070274533,
  4531940473,
  45632103,
  865622

In [75]:
friend_ids = f[2]

In [76]:
# paginate -> make the next API call

f = api.GetFriendIDsPaged(screen_name=sn, cursor=f[0])

In [77]:
f

(-1, -1, [])

In [78]:
# we can do this a few times -> paginate through friends list!
# in my case, there's no need

#f = api.GetFriendIDsPaged(screen_name=sn, cursor=f[0])
#friend_ids += f[2]
#len(friend_ids)

In [79]:
# now lets get user info

friend_ids[0]
ans = api.GetUser(user_id=friend_ids[0])

In [80]:
ans.screen_name

u'kevinstownsend'

In [81]:
ans.AsDict()

{u'created_at': u'Thu Oct 14 05:18:32 +0000 2010',
 u'description': u'Word+Data Journalist \u2022 @Slate @BuzzFeed \u2022 @Stanford + @Columbia \u2022 GPG: 78C8 FCFB D3F3 7296 0C8D 5666 3744 5357 1F70 1DC8',
 u'favourites_count': 1750,
 u'followers_count': 771,
 u'following': True,
 u'friends_count': 627,
 u'geo_enabled': True,
 u'id': 202503479,
 u'lang': u'en',
 u'listed_count': 39,
 u'location': u'Washington, DC',
 u'name': u'Kevin Townsend',
 u'profile_background_color': u'9AE4E8',
 u'profile_background_image_url': u'http://pbs.twimg.com/profile_background_images/628897612/ytry1i6xfxamrzsmh43r.jpeg',
 u'profile_background_tile': True,
 u'profile_banner_url': u'https://pbs.twimg.com/profile_banners/202503479/1398233971',
 u'profile_image_url': u'http://pbs.twimg.com/profile_images/2512365434/tqv64998hyrlv6zgyx80_normal.jpeg',
 u'profile_link_color': u'0084B4',
 u'profile_sidebar_fill_color': u'DAECF4',
 u'profile_text_color': u'663B12',
 u'screen_name': u'kevinstownsend',
 u'status'

In [82]:
# rate limit

resource_url = 'https://api.twitter.com/1.1/users/show.json'
api.rate_limit.get_limit(resource_url)

EndpointRateLimit(limit=900, remaining=899, reset=1480909765)

In [83]:
a = api.rate_limit.get_limit(resource_url)
a.remaining

899

In [ ]:
user_info={}

for u_id in friend_ids:
    
    # check rate limiting
    remaining = api.rate_limit.get_limit(resource_url).remaining
    if remaining == 0:
        break
        
    # check if we already have this user's info, continue onto the next iteration
    if u_id in user_info.keys():
        continue
        
    # otherwise - get this user's info
    user_info[u_id] = api.GetUser(user_id=u_id)
    print 'got user', user_info[u_id].screen_name
    
    # print
    if len(user_info) % 10 == 0:
        print len(user_info)

In [85]:
len(user_info)

899

In [95]:
import networkx as nx

# Create Graph

g = nx.DiGraph()

In [96]:
g.add_nodes_from([(u_id, {'label':u.screen_name, 'followers':u.followers_count}) for u_id,u in user_info.items()])

In [112]:
g.node[705113652471439361]

{'followers': 28455, 'label': u'DeepDrumpf'}

In [113]:
g.node[705113652471439361]['label']

u'DeepDrumpf'

In [123]:
# lets see which edges we can add for ITP

cur_user = 'ITP_NYU'
cur_id = 65410718
follows = api.GetFriendIDsPaged(user_id=cur_id)

# save the results in a dict
all_edges = {}
all_edges[cur_id]=follows

In [126]:
# out of all the users that @ITP_NYU follows, lets add edges only to users who exist
# as nodes in our graph

follow_intersection = [x for x in follows[2] if x in g.nodes()]
follow_intersection

[1536791610,
 266060880,
 111057129,
 19605305,
 23677076,
 373729837,
 36443,
 20338182,
 8112862,
 627683742,
 5454152,
 17653842,
 2384071,
 48048511,
 935447275,
 388077354,
 7482,
 14338436,
 817409,
 768632,
 12997952,
 190807773]

In [127]:
for u_id in follow_intersection:
    
    # add edge from @ITP_NYU (remember: directed graph)
    g.add_edge(cur_id, u_id)
    

In [129]:
# we have a handful of edges

g.edges()

[(65410718, 817409),
 (65410718, 14338436),
 (65410718, 20338182),
 (65410718, 23677076),
 (65410718, 627683742),
 (65410718, 388077354),
 (65410718, 373729837),
 (65410718, 19605305),
 (65410718, 1536791610),
 (65410718, 7482),
 (65410718, 12997952),
 (65410718, 2384071),
 (65410718, 5454152),
 (65410718, 266060880),
 (65410718, 17653842),
 (65410718, 36443),
 (65410718, 190807773),
 (65410718, 8112862),
 (65410718, 111057129),
 (65410718, 935447275),
 (65410718, 768632),
 (65410718, 48048511)]

In [197]:
# rate limiting

resource_url = 'https://api.twitter.com/1.1/friends/ids.json'
api.rate_limit.get_limit(resource_url)

EndpointRateLimit(limit=15, remaining=15, reset=0)

In [198]:
from datetime import datetime
print datetime.fromtimestamp(1480913860)

2016-12-04 23:57:40


In [ ]:
remaining = api.rate_limit.get_limit(resource_url).remaining

while remaining > 0:
    
    # pick a node
    n = random.choice(g.nodes())
    while n in all_edges.keys():
        # pick different node if we already have its info
        n = random.choice(g.nodes())
            
    # get friend info for n
    follows = api.GetFriendIDsPaged(user_id=n)

    # save the results in a dict
    all_edges = {}
    all_edges[n]=follows

    follow_intersection = [x for x in follows[2] if x in g.nodes()]
    print 'intersection:', len(follow_intersection)
    
    for u_id in follow_intersection:
        g.add_edge(n, u_id)
        
    remaining = api.rate_limit.get_limit(resource_url).remaining
    print 'remaining:',remaining
    

In [189]:
nx.write_gexf(g, 'data/gilgul.gexf')